In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

Cargar archivo archivo kaggle.json

In [4]:
from google.colab import files
files.upload()  # selecciona el archivo kaggle.json que descargaste

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yeissondavidmanrique","key":"bd4b5247f127828c2f5033e46d264eeb"}'}

In [5]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Descargar los archivos de la Competencia

In [6]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


Descomprimir los archivos de la Competencia

In [7]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submission_example.csv  
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test.csv                
  inflating: train.csv               


In [8]:
# Carga el archivo de entrenamiento
t = pd.read_csv("test.csv")

In [9]:
# Muestra la forma (número de filas y columnas)
print("shape of loaded dataframe", t.shape)

shape of loaded dataframe (296786, 20)


In [10]:
# Muestra las primeras filas
t.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa,0.328,0.219,0.317,0.247
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa,0.227,0.283,0.296,0.324
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.285,0.228,0.294,0.247
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta,0.160,0.408,0.217,0.294
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,0.209,0.283,0.306,0.286


In [11]:
# Carga el archivo de entrenamiento
z = pd.read_csv("train.csv")

In [12]:
# Muestra la forma (número de filas y columnas)
print("shape of loaded dataframe", z.shape)

shape of loaded dataframe (692500, 21)


In [13]:
# Muestra las primeras filas
z.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [14]:
z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 non-null  object 
 13 

In [15]:
# Muestra la suma de valores faltantes por columna
k = z.isna().sum()
print(k[k != 0])

E_VALORMATRICULAUNIVERSIDAD     6287
E_HORASSEMANATRABAJA           30857
F_ESTRATOVIVIENDA              32137
F_TIENEINTERNET                26629
F_EDUCACIONPADRE               23178
F_TIENELAVADORA                39773
F_TIENEAUTOMOVIL               43623
E_PAGOMATRICULAPROPIO           6498
F_TIENECOMPUTADOR              38103
F_TIENEINTERNET.1              26629
F_EDUCACIONMADRE               23664
dtype: int64


Limpieza y preprocesado de datos

In [16]:
# Mapeo de las categorías de rendimiento a valores discretos (enteros)
rmap = { 'alto': 3, 'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2 }

In [18]:
y_col = 'RENDIMIENTO_GLOBAL'
z[y_col] = [rmap[i] for i in z[y_col]]


KeyError: 2

In [19]:
z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 non-null  object 
 13 

In [20]:
# Muestra la suma de valores faltantes por columna
k = z.isna().sum()
print(k[k != 0])

E_VALORMATRICULAUNIVERSIDAD     6287
E_HORASSEMANATRABAJA           30857
F_ESTRATOVIVIENDA              32137
F_TIENEINTERNET                26629
F_EDUCACIONPADRE               23178
F_TIENELAVADORA                39773
F_TIENEAUTOMOVIL               43623
E_PAGOMATRICULAPROPIO           6498
F_TIENECOMPUTADOR              38103
F_TIENEINTERNET.1              26629
F_EDUCACIONMADRE               23664
dtype: int64


In [21]:
z.F_EDUCACIONMADRE.values[z.F_EDUCACIONMADRE.isna()] = 'no info'

In [22]:
z['F_EDUCACIONMADRE'].value_counts()

,count
F_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,141744
Primaria incompleta,99420
Técnica o tecnológica completa,89542
Educación profesional completa,85326
Secundaria (Bachillerato) incompleta,81012
Primaria completa,56125
Postgrado,46246
Técnica o tecnológica incompleta,27533
no info,23664


In [23]:
z.F_EDUCACIONPADRE.values[z.F_EDUCACIONPADRE.isna()] = 'no info'
t.F_EDUCACIONPADRE.values[t.F_EDUCACIONPADRE.isna()] = 'no info'

In [24]:
z.E_VALORMATRICULAUNIVERSIDAD.values[z.E_VALORMATRICULAUNIVERSIDAD.isna()] = 'no info'
t.E_VALORMATRICULAUNIVERSIDAD.values[t.E_VALORMATRICULAUNIVERSIDAD.isna()] = 'no info'

In [25]:
z.E_HORASSEMANATRABAJA.values[z.E_HORASSEMANATRABAJA.isna()] = 'no info'
t.E_HORASSEMANATRABAJA.values[t.E_HORASSEMANATRABAJA.isna()] = 'no info'

In [26]:
hours_map = {
    '0': 0.0,
    'Menos de 10 horas': 5.0,        # Punto medio aproximado
    'Entre 11 y 20 horas': 15.5,     # Valor representativo
    'Entre 21 y 30 horas': 25.5,     # Valor representativo
    'Más de 30 horas': 35.0,         # Un valor alto representativo
    'no info': -1.0                  # Valor distintivo para datos imputados
}

In [ ]:
z['E_HORASSEMANATRABAJA'] = np.r_[[hours_map[i] for i in z.E_HORASSEMANATRABAJA]]
t['E_HORASSEMANATRABAJA'] = np.r_[[hours_map[i] for i in t.E_HORASSEMANATRABAJA]]

In [27]:
# 1. IMPUTACIÓN (se asume que z es el DataFrame y se manejan NaNs)
z.F_TIENEINTERNET.values[z.F_TIENEINTERNET.isna()] = 'no info'
# 1. IMPUTACIÓN (se asume que z es el DataFrame y se manejan NaNs)
t.F_TIENEINTERNET.values[t.F_TIENEINTERNET.isna()] = 'no info'

In [28]:
# 2. MAPEO BINARIO (Creación del diccionario de conversión)
# Se utiliza el mapeo a 0 o 1.
internet_map = {
    'Si': 1,
    'No': 0,
    'no info': -1 # Se asigna un valor distintivo a los imputados si se aplica
}


In [29]:
# Aplicar el mapeo a la columna:
z['F_TIENEINTERNET'] = np.r_[[internet_map[i] for i in z.F_TIENEINTERNET]]
# Aplicar el mapeo a la columna:
t['F_TIENEINTERNET'] = np.r_[[internet_map[i] for i in t.F_TIENEINTERNET]]

In [30]:
# Sustituir NaN con 'no info' (análogo a lo realizado en [4, 5])
z.E_PAGOMATRICULAPROPIO.values[z.E_PAGOMATRICULAPROPIO.isna()] = 'no info'
# Sustituir NaN con 'no info' (análogo a lo realizado en [4, 5])
t.E_PAGOMATRICULAPROPIO.values[t.E_PAGOMATRICULAPROPIO.isna()] = 'no info'

In [31]:
pay_map = {
    'Si': 1,
    'No': 0,
    'no info': -1  # Valor para datos faltantes/imputados
}


In [32]:
# Aplicar el mapeo a la columna para convertirla a un array numérico [4, 6]
z['E_PAGOMATRICULAPROPIO'] = np.r_[[pay_map[i] for i in z.E_PAGOMATRICULAPROPIO]]
# Aplicar el mapeo a la columna para convertirla a un array numérico [4, 6]
t['E_PAGOMATRICULAPROPIO'] = np.r_[[pay_map[i] for i in t.E_PAGOMATRICULAPROPIO]]

In [33]:
# Se sustituyen los valores faltantes (NaN) por 'no info'
z.F_TIENECOMPUTADOR.values[z.F_TIENECOMPUTADOR.isna()] = 'no info'
t.F_TIENECOMPUTADOR.values[t.F_TIENECOMPUTADOR.isna()] = 'no info'

In [34]:
computer_map = {
    'Si': 1,
    'No': 0,
    'no info': -1  # Valor para datos faltantes/imputados
}

In [35]:
# Aplicar el mapeo a la columna para convertirla a un array numérico:
# La sintaxis np.r_ y el uso de un diccionario para el mapeo es común para estas transformaciones [2].
z['F_TIENECOMPUTADOR'] = np.r_[[computer_map[i] for i in z.F_TIENECOMPUTADOR]]
t['F_TIENECOMPUTADOR'] = np.r_[[computer_map[i] for i in t.F_TIENECOMPUTADOR]]

In [36]:
# Eliminación de Variables no relevantes
z = z.drop('F_TIENELAVADORA', axis=1)
z = z.drop('F_TIENEAUTOMOVIL', axis=1)
t = t.drop('F_TIENELAVADORA', axis=1)
t = t.drop('F_TIENEAUTOMOVIL', axis=1)

In [ ]:
# 6. Construir matrices X y y
# Asegurarse de que las columnas están ordenadas para reproducibilidad [12].
#z = z[sorted(z.columns)]
#X = z[[c for c in z.columns if c != y_col]].values
#y = z[y_col].values


In [37]:
# 1. Definir la variable objetivo
y = z["RENDIMIENTO_GLOBAL"]     # <-- Cambia si tu target es otro
X = z.drop("RENDIMIENTO_GLOBAL", axis=1)

# 2. Train/Test split
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Detectar columnas categóricas y numéricas
cat_cols = Xtr.select_dtypes(include=['object']).columns
num_cols = Xtr.select_dtypes(exclude=['object']).columns

# 4. Preprocesamiento
preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('num', 'passthrough', num_cols)
    ]
)



In [38]:
# 5. Pipeline con RandomForest
model = Pipeline(steps=[
    ('prep', preprocess),
    ('rf', RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42
    ))
])

# 6. Entrenar
model.fit(Xtr, ytr)

print("Modelo entrenado correctamente ✔")

Modelo entrenado correctamente ✔


In [39]:
# 9. Predicción y Evaluación
# Generar predicciones para los conjuntos de entrenamiento y prueba
preds_tr = model.predict(Xtr)
preds_ts = model.predict(Xte)

In [41]:
# Calcular la precisión (accuracy) [17]
accuracy_train = np.mean(preds_tr == ytr)
accuracy_test = np.mean(preds_ts == yte)

In [42]:
print(f"\nAccuracy del Random Forest en Train: {accuracy_train:.4f}")
print(f"Accuracy del Random Forest en Test: {accuracy_test:.4f}")


Accuracy del Random Forest en Train: 0.4001
Accuracy del Random Forest en Test: 0.3939


In [47]:
pred = model.predict(t)

In [48]:
pred_df = pd.DataFrame(pred, columns=["PREDICCION"])
pred_df.head()

,PREDICCION
0,3
1,2
2,3
3,0
4,1


In [49]:
submission = pd.DataFrame({
    "ID": t["ID"],
    "RENDIMIENTO_GLOBAL": pred
})

submission.to_csv("submission.csv", index=False)

In [52]:
# Mapeo original
i_rmap = {
    'alto': 3,
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2
}

# Crear diccionario invertido
rmap_inv = {v: k for k, v in i_rmap.items()}

# Revertir columna
y_col = 'RENDIMIENTO_GLOBAL'
t[y_col] = [rmap_inv[i] for i in t[y_col]]

KeyError: 'RENDIMIENTO_GLOBAL'

In [53]:
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,3
1,98545,2
2,499179,3
3,782980,0
4,785185,1
